In [ ]:
#content_filtering code

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load dataset
data = pd.read_excel("sample_data.xlsx")

# Preprocessing
# Encode categorical variables
label_encoders = {}
for col in ["state", "category"]:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

# Split data
X = data.drop(["place1", "place2", "place3"], axis=1)
y = data[["place1", "place2", "place3"]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train model (example using RandomForestClassifier)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# User input (state, category, rating)
user_input_state = "Uttarakhand"  # Replace with actual user input
user_input_category = "Religious"
user_input_rating = 4.5

# Process user input
user_input_encoded = pd.DataFrame({
    "state": [label_encoders["state"].transform([user_input_state])[0]],
    "category": [label_encoders["category"].transform([user_input_category])[0]],
    "rating": [user_input_rating]
})

# Get recommendations
predictions = model.predict(user_input_encoded)

print("Top 3 recommended places:", predictions)


Top 3 recommended places: [['Haridwar' ' Panch Kedar' ' Hemkund Sahib']]


In [ ]:
#Content_filtering_using_tensorflow

In [38]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import tensorflow as tf
import numpy as np

# Load the data
data = pd.read_excel('sample_data.xlsx')

# Initialize label encoders for categorical features and outputs
label_encoders = {}
for column in ['state', 'category', 'place1', 'place2', 'place3']:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le

# Normalize the rating column
scaler = MinMaxScaler()
data['rating'] = scaler.fit_transform(data[['rating']])

# Separate features and outputs
X = data[['state', 'category', 'rating']].values
y1 = data['place1'].values
y2 = data['place2'].values
y3 = data['place3'].values

# Convert outputs to one-hot encoding
y1 = tf.keras.utils.to_categorical(y1, num_classes=len(label_encoders['place1'].classes_))
y2 = tf.keras.utils.to_categorical(y2, num_classes=len(label_encoders['place2'].classes_))
y3 = tf.keras.utils.to_categorical(y3, num_classes=len(label_encoders['place3'].classes_))

# Input shape
input_shape = X.shape[1]

# Define the model
input_layer = tf.keras.layers.Input(shape=(input_shape,))
dense_layer = tf.keras.layers.Dense(64, activation='relu')(input_layer)
dense_layer = tf.keras.layers.Dense(64, activation='relu')(dense_layer)

# Output layers for place1, place2, and place3
output1 = tf.keras.layers.Dense(len(label_encoders['place1'].classes_), activation='softmax')(dense_layer)
output2 = tf.keras.layers.Dense(len(label_encoders['place2'].classes_), activation='softmax')(dense_layer)
output3 = tf.keras.layers.Dense(len(label_encoders['place3'].classes_), activation='softmax')(dense_layer)

# Create the model
model = tf.keras.models.Model(inputs=input_layer, outputs=[output1, output2, output3])

# Compile the model with separate metrics for each output
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy', 'accuracy', 'accuracy'])

# Train the model
model.fit(X, [y1, y2, y3], epochs=150, batch_size=32, validation_split=0.2)

# Function to preprocess user input
def preprocess_input(state, category, rating):
    state_encoded = label_encoders['state'].transform([state])
    category_encoded = label_encoders['category'].transform([category])
    rating_normalized = scaler.transform([[rating]])
    return np.array([state_encoded[0], category_encoded[0], rating_normalized[0][0]])

# Function to predict and decode recommendations
def recommend_places(state, category, rating):
    input_data = preprocess_input(state, category, rating)
    input_data = np.expand_dims(input_data, axis=0)
    predictions = model.predict(input_data)
    place1 = label_encoders['place1'].inverse_transform([np.argmax(predictions[0])])[0]
    place2 = label_encoders['place2'].inverse_transform([np.argmax(predictions[1])])[0]
    place3 = label_encoders['place3'].inverse_transform([np.argmax(predictions[2])])[0]
    return place1, place2, place3

# Example usage
state = 'Uttarakhand'
category = 'Hill Station'
rating = 4.0
recommendations = recommend_places(state, category, rating)
print(f'Recommended places: {recommendations}')



Epoch 1/150
44/44 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - dense_77_accuracy: 0.0096 - dense_78_accuracy: 0.0298 - dense_79_accuracy: 0.0256 - loss: 13.4359 - val_dense_77_accuracy: 0.1136 - val_dense_78_accuracy: 0.0767 - val_dense_79_accuracy: 0.1364 - val_loss: 12.5596
Epoch 2/150
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - dense_77_accuracy: 0.0978 - dense_78_accuracy: 0.0672 - dense_79_accuracy: 0.1109 - loss: 12.4705 - val_dense_77_accuracy: 0.1420 - val_dense_78_accuracy: 0.0938 - val_dense_79_accuracy: 0.1619 - val_loss: 11.2408
Epoch 3/150
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - dense_77_accuracy: 0.0975 - dense_78_accuracy: 0.0662 - dense_79_accuracy: 0.1213 - loss: 11.4205 - val_dense_77_accuracy: 0.1875 - val_dense_78_accuracy: 0.1335 - val_dense_79_accuracy: 0.1932 - val_loss: 9.8574
Epoch 4/150
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - dense_77_accuracy: 0.1649 - dense_78_accuracy: 0.0892 - dense_79_accuracy: 0.1648 - loss: 10.1836 - val_dense_77_accuracy: 0.2585 - val_dense_78_a

C:\Users\h8888\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step
Recommended places: ('Haldwani', ' Pauri', ' Bhowali')


In [20]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import tensorflow as tf
import numpy as np

# Load the data
data = pd.read_excel('sample_data.xlsx')

# Initialize label encoders for categorical features and outputs
label_encoders = {}
for column in ['state', 'category', 'place1', 'place2', 'place3']:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le

# Normalize the rating column
scaler = MinMaxScaler()
data['rating'] = scaler.fit_transform(data[['rating']])

# Separate features and outputs
X = data[['state', 'category', 'rating']].values
y1 = data['place1'].values
y2 = data['place2'].values
y3 = data['place3'].values

# Convert outputs to one-hot encoding
y1 = tf.keras.utils.to_categorical(y1, num_classes=len(label_encoders['place1'].classes_))
y2 = tf.keras.utils.to_categorical(y2, num_classes=len(label_encoders['place2'].classes_))
y3 = tf.keras.utils.to_categorical(y3, num_classes=len(label_encoders['place3'].classes_))

# Input shape
input_shape = X.shape[1]

# Define the model
input_layer = tf.keras.layers.Input(shape=(input_shape,))
dense_layer = tf.keras.layers.Dense(64, activation='relu')(input_layer)
dense_layer = tf.keras.layers.Dense(64, activation='relu')(dense_layer)

# Output layers for place1, place2, and place3
output1 = tf.keras.layers.Dense(len(label_encoders['place1'].classes_), activation='softmax')(dense_layer)
output2 = tf.keras.layers.Dense(len(label_encoders['place2'].classes_), activation='softmax')(dense_layer)
output3 = tf.keras.layers.Dense(len(label_encoders['place3'].classes_), activation='softmax')(dense_layer)

# Create the model
model = tf.keras.models.Model(inputs=input_layer, outputs=[output1, output2, output3])

# Compile the model with separate metrics for each output
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy', 'accuracy', 'accuracy'])

# Train the model
model.fit(X, [y1, y2, y3], epochs=150, batch_size=32, validation_split=0.2)

# Save the model with .h5 extension
model.save('my_model.h5')

# Load the model back from the .h5 file
loaded_model = tf.keras.models.load_model('my_model.h5')

# Define the input signature
input_signature = [tf.TensorSpec(shape=(None, input_shape), dtype=tf.float32)]

# Create a concrete function from the Keras model
concrete_func = tf.function(lambda x: loaded_model(x))
concrete_func = concrete_func.get_concrete_function(input_signature)

# Convert the loaded model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
tflite_model = converter.convert()

# Save the TensorFlow Lite model to a file
with open('my_model.tflite', 'wb') as f:
    f.write(tflite_model)

# Function to preprocess user input
def preprocess_input(state, category, rating):
    state_encoded = label_encoders['state'].transform([state])
    category_encoded = label_encoders['category'].transform([category])
    rating_normalized = scaler.transform([[rating]])
    return np.array([state_encoded[0], category_encoded[0], rating_normalized[0][0]])

# Function to predict and decode recommendations
def recommend_places(state, category, rating):
    input_data = preprocess_input(state, category, rating)
    input_data = np.expand_dims(input_data, axis=0)
    predictions = loaded_model.predict(input_data)
    place1 = label_encoders['place1'].inverse_transform([np.argmax(predictions[0])])[0]
    place2 = label_encoders['place2'].inverse_transform([np.argmax(predictions[1])])[0]
    place3 = label_encoders['place3'].inverse_transform([np.argmax(predictions[2])])[0]
    return place1, place2, place3

# Example usage
state = 'Delhi'
category = 'Wildlife'
rating = 4.5
recommendations = recommend_places(state, category, rating)
print(f'Recommended places: {recommendations}')


Epoch 1/150
44/44 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - dense_2_accuracy: 0.0141 - dense_3_accuracy: 0.0390 - dense_4_accuracy: 0.0106 - loss: 13.4208 - val_dense_2_accuracy: 0.1023 - val_dense_3_accuracy: 0.0597 - val_dense_4_accuracy: 0.0966 - val_loss: 12.4071
Epoch 2/150
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - dense_2_accuracy: 0.1050 - dense_3_accuracy: 0.0635 - dense_4_accuracy: 0.0873 - loss: 12.2861 - val_dense_2_accuracy: 0.1307 - val_dense_3_accuracy: 0.1335 - val_dense_4_accuracy: 0.1648 - val_loss: 11.1212
Epoch 3/150
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - dense_2_accuracy: 0.1515 - dense_3_accuracy: 0.0977 - dense_4_accuracy: 0.1545 - loss: 11.1121 - val_dense_2_accuracy: 0.2131 - val_dense_3_accuracy: 0.1477 - val_dense_4_accuracy: 0.1932 - val_loss: 9.6012
Epoch 4/150
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - dense_2_accuracy: 0.2074 - dense_3_accuracy: 0.1204 - dense_4_accuracy: 0.1546 - loss: 9.9379 - val_dense_2_accuracy: 0.2443 - val_dense_3_accuracy: 0.1705 - val_de

C:\Users\h8888\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step
Recommended places: ('National Zoological Park', 'Okhla Bird Sanctuary', 'Yamuna biodiversity Park')


In [4]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import tensorflow as tf
import numpy as np

# Load the data
data = pd.read_excel('sample_data.xlsx')

# Initialize label encoders for categorical features and outputs
label_encoders = {}
encoded_values = {}

for column in ['state', 'category', 'place1', 'place2', 'place3']:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le
    encoded_values[column] = le.classes_

# Normalize the rating column
scaler = MinMaxScaler()
data['rating'] = scaler.fit_transform(data[['rating']])

# Separate features and outputs
X = data[['state', 'category', 'rating']].values
y1 = data['place1'].values
y2 = data['place2'].values
y3 = data['place3'].values

# Print the encoding numbers of all distinct categories
for column in encoded_values:
    print(f"Encoding for {column}:")
    for idx, value in enumerate(encoded_values[column]):
        print(f"{value} -> {idx}")
    print()


Encoding for state:
Delhi -> 0
Uttarakhand -> 1

Encoding for category:
Adventure sports -> 0
Culture -> 1
Entertainment -> 2
Hill Station -> 3
Historical -> 4
Mall -> 5
Market -> 6
Museum -> 7
Natural Beauty -> 8
Religious -> 9
Wildlife -> 10

Encoding for place1:
Akshardham Temple -> 0
Ambience Mall -> 1
Appu Ghar Express -> 2
Askot Sanctuary -> 3
Auli -> 4
Badrinath -> 5
Bangla Sahib Market -> 6
Bhimtal lake -> 7
Bhulla Lake -> 8
Chakrata -> 9
Chamoli -> 10
Chandni Chowk -> 11
Chhatarpur Temple -> 12
Chopta -> 13
Connaught Place -> 14
Corbett National Park -> 15
DLF CyberHub -> 16
DLF Emporio -> 17
Dehradun -> 18
Delhi Haunted House -> 19
Delhi Ridge Forest -> 20
Dilli Haat INA -> 21
Dronagiri -> 22
Gandhi Smriti -> 23
George Everest Peak -> 24
Gurudwara Bangla Sahib -> 25
Haldwani -> 26
Haridwar -> 27
Hauz Khas Village -> 28
Humayun’s Tomb -> 29
ISKCON Temple -> 30
India Gate -> 31
India Habitat Centre -> 32
India International Centre -> 33
Jageshwar -> 34
Jama Masjid -> 35
Janpath

In [26]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

loaded_model = tf.keras.models.load_model('my_model.h5')

def preprocess_input(state, category, rating):
    state_encoded = label_encoders['state'].transform([state])
    category_encoded = label_encoders['category'].transform([category])
    rating_normalized = scaler.transform([[rating]])
    print( state_encoded, category_encoded, type(rating_normalized))
    print( state_encoded[0], category_encoded[0], type(rating_normalized[0][0]))
    return np.array([state_encoded[0], category_encoded[0], rating_normalized[0][0]])

# Function to predict and decode recommendations
def recommend_places(state, category, rating):
    input_data = preprocess_input(state, category, rating)
    input_data = np.expand_dims(input_data, axis=0)
    predictions = loaded_model.predict(input_data)
    print("Type of predictions:", type(predictions))
    print("Length of predictions:", len(predictions))
    print("Predictions:", predictions)
    
    # Assuming predictions is a list of arrays, print the shape of each array
    for i, pred in enumerate(predictions):
        print(f"Shape of predictions[{i}]:", pred.shape)
    place1 = label_encoders['place1'].inverse_transform([np.argmax(predictions[0])])[0]
    place2 = label_encoders['place2'].inverse_transform([np.argmax(predictions[1])])[0]
    place3 = label_encoders['place3'].inverse_transform([np.argmax(predictions[2])])[0]
    return place1, place2, place3

# Example usage
state = 'Uttarakhand'
category = 'Religious'
rating = 4.5
recommendations = recommend_places(state, category, rating)
print(f'Recommended places: {recommendations}')


C:\Users\h8888\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


[1] [9] <class 'numpy.ndarray'>
1 9 <class 'numpy.float64'>
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
Type of predictions: <class 'list'>
Length of predictions: 3
Predictions: [array([[2.18224450e-04, 1.46623558e-08, 4.11087536e-15, 4.22247202e-11,
        8.22213393e-18, 1.63017154e-01, 1.08422732e-07, 6.05930763e-05,
        3.50890950e-05, 5.57536376e-04, 3.98355944e-08, 1.76552961e-09,
        1.13450904e-12, 6.18832605e-03, 3.45844064e-09, 2.42803898e-03,
        8.78306505e-15, 6.44992397e-08, 4.94163217e-08, 9.44858149e-24,
        3.20166555e-13, 1.69184800e-09, 1.79656171e-12, 2.65577977e-08,
        1.48694623e-09, 1.05811436e-04, 1.67736980e-09, 4.33491290e-01,
        1.81560155e-23, 8.38707749e-12, 9.20159073e-05, 5.55895336e-13,
        1.37933192e-29, 7.68839388e-22, 2.57644133e-05, 3.76329935e-13,
        2.60887685e-08, 1.11007698e-12, 5.11212202e-05, 3.99538694e-05,
        6.51796436e-05, 9.06460613e-08, 3.26277852e-01, 1.59913629e-08,
        1.41332393e-05, 6.88186503

In [14]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import tensorflow as tf
import numpy as np

# Load the data
data = pd.read_excel('sample_data.xlsx')

# Initialize label encoders for categorical features and outputs
label_encoders = {}
for column in ['state', 'category', 'place1', 'place2', 'place3']:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le
    print(data[column])

0       0
1       0
2       0
3       0
4       0
       ..
1751    1
1752    1
1753    1
1754    1
1755    1
Name: state, Length: 1756, dtype: int32
0        1
1        1
2        2
3        2
4        2
        ..
1751    11
1752    11
1753    11
1754    11
1755    11
Name: category, Length: 1756, dtype: int32
0       33
1       32
2       16
3       16
4       16
        ..
1751    15
1752    15
1753    15
1754    68
1755     3
Name: place1, Length: 1756, dtype: int32
0       70
1       76
2        1
3        1
4       36
        ..
1751    54
1752    54
1753    51
1754    51
1755    53
Name: place2, Length: 1756, dtype: int32
0       90
1       85
2       44
3       76
4       76
        ..
1751    57
1752     8
1753     8
1754     8
1755    79
Name: place3, Length: 1756, dtype: int32
